In [31]:
import boto3
import json
import io
import gzip
from io import BytesIO, TextIOWrapper
import yaml
from datetime import datetime
import pandas as pd

In [38]:
# s3 bucket holding our cloudfront logs
S3_BUCKET='s3-bucket-with-cloudfront-logs'
# the date (or substring) of the log files we are interested in
substring="2020-04-1"

In [39]:
# assumes you have a sane aws environment and perms to access your target bucket
s3_client = boto3.client('s3')

In [40]:
# the resulting data frame 
df=None
# list of s3 keys
keys=[]

print(f'started at {datetime.utcnow()}')
paginator = s3_client.get_paginator('list_objects_v2')

# iterate keys matching the file/key name substring
page_iterator = paginator.paginate(
    Bucket=S3_BUCKET,
    Prefix="")
for item in page_iterator.search(f"Contents[?contains(Key, `{substring}`)][]"):
    if not item['Key'].endswith('/'):    
        keys.append(item['Key'])   

# enumerate keys and add content to a data frame
# assumes the header in all are the same
for i,key in enumerate(keys[:]):
    # debug if you want to watch along
    # print(key)
    content=""
    s3_response = s3_client.get_object(Bucket=S3_BUCKET, Key=key)
    s3_raw_data = s3_response["Body"].read()
    if key[-3:] == ".gz":
        with gzip.GzipFile(fileobj=BytesIO(s3_raw_data)) as gzip_stream:
            content += "".join(TextIOWrapper(gzip_stream,encoding='utf-8'))
    else:    
        content=s3_response["Body"].read().decode('utf-8')

    # get column names
    names=list(pd.read_csv(io.StringIO(content),sep=' ',lineterminator='\n',skiprows=1, nrows=0))
    # get rid of the '#Fields:' errant column
    names.pop(0)
    if i==0:
        # initalize the data from from the first s3 key
        df=pd.read_csv(io.StringIO(content), sep='\t', lineterminator='\n',skiprows=(0,1),names=names)
    else:
        # append this key's contents to the dataframe we are building
        df_csv=pd.read_csv(io.StringIO(content), sep='\t', lineterminator='\n',skiprows=(0,1),names=names)
        # debug if you are interested in knowing how many records in each file
        # print(f"appending {len(df_csv)} records")
        df=df.append(df_csv)

print(f'ended at {datetime.utcnow()}')
print(f'{len(df)} records loaded')

started at 2020-04-20 23:07:51.133946
ended at 2020-04-20 23:08:22.510581
1670 records loaded


In [ ]:
df.head()

In [8]:
df.columns.to_list()

['date',
 'time',
 'x-edge-location',
 'sc-bytes',
 'c-ip',
 'cs-method',
 'cs(Host)',
 'cs-uri-stem',
 'sc-status',
 'cs(Referer)',
 'cs(User-Agent)',
 'cs-uri-query',
 'cs(Cookie)',
 'x-edge-result-type',
 'x-edge-request-id',
 'x-host-header',
 'cs-protocol',
 'cs-bytes',
 'time-taken',
 'x-forwarded-for',
 'ssl-protocol',
 'ssl-cipher',
 'x-edge-response-result-type',
 'cs-protocol-version',
 'fle-status',
 'fle-encrypted-fields',
 'c-port',
 'time-to-first-byte',
 'x-edge-detailed-result-type',
 'sc-content-type',
 'sc-content-len',
 'sc-range-start',
 'sc-range-end']

In [9]:
# Fix up column names to be more friendly
df.rename(columns={'cs(Referer)': 'referer'}, inplace=True)
#replace any other ones with () to dashes
df.columns = df.columns.str.replace('(','-')
df.columns = df.columns.str.replace(')','')
df.columns = df.columns.str.lower()

In [45]:
df['cs-uri-stem'].describe()

count     2916
unique     204
top          /
freq       757
Name: cs-uri-stem, dtype: object

In [ ]:
# top 10 uris
df['cs-uri-stem'].value_counts().head(10)

In [47]:
# bottom 10 uris
df['cs-uri-stem'].value_counts().tail(10)

/wp-config.php_                   1
/Fckeditor/editor/                1
/admin/Fckeditor/editor/          1
/wp-config.bak                    1
/apple-touch-icon-152x152.png     1
/fckeditor/editor/                1
/FCKeditor/editor/                1
/contact.html/js/popper.min.js    1
/wp-config.php.swp                1
/wp-admin/admin-ajax.php          1
Name: cs-uri-stem, dtype: int64

In [48]:
len(df[ df['cs-user-agent'].str.contains('bot') ])

533

In [49]:
# 200 OK
len(df[df['sc-status'].isin([200])])

1628

In [50]:
# not OK
len(df[~df['sc-status'].isin([200])])

1288

In [10]:
# setup columns for easy aliasing of data points
# true/false bot status
df['bot']=df['cs-user-agent'].str.contains('bot')

# true false ok
df['ok']=df['sc-status'].isin([200])

In [52]:
# referer counts
df[ (df['cs-uri-stem']=='/' ) & (df['ok']) & (df['bot']==False)]['referer'].value_counts()

-                                                  353
https://facebookfracking.watch/                     10
http://facebookfracking.watch/                       5
https://www.google.com/                              3
https://facebookfracking.watch/updates.html          2
http://facebookfrac-out.watch/                       2
http://facebookfrac-out.watch                        2
https://facebookfracking.watch/contact.html          2
https://facebookfrac-out.watch/                      2
https://houston.moreover.com/houston/index.html      2
https://t.co/uHfvFRtJ0Z                              2
http://facebookdrilling.watch/                       2
http://facebookdrilling.watch                        1
http://facebookfracking.watch                        1
Name: referer, dtype: int64

In [53]:
# what's popular that's not a bot, not ok
df[(df['bot']==False) & (df['ok']==False)]['cs-uri-stem'].value_counts()

/                                            267
/robots.txt                                  112
/wp-login.php                                 47
/admin/                                       18
/ads.txt                                      12
                                            ... 
/contact.html/js/jquery-3.3.1.slim.min.js      1
/wp-config.php.new                             1
/wp-config.php_orig                            1
/apple-touch-icon-152x152-precomposed.png      1
/admin/Fckeditor/editor/                       1
Name: cs-uri-stem, Length: 178, dtype: int64

In [55]:
# find the IP for wordpress probers
wp_hackers = df.loc[(df['cs-uri-stem'].str.contains('wp'))].to_dict('records')
for record in wp_hackers[:10]:
    print(record['c-ip'])

157.245.82.57
157.245.82.57
195.154.170.245
195.154.170.245
195.154.170.245
195.154.170.245
195.154.170.245
195.154.170.245
195.154.170.245
195.154.170.245


In [ ]:
#common user agents
df.loc[df['bot']]['cs-user-agent'].value_counts()

In [57]:
#common hits that are not ok/200
df[~df['ok']]['cs-uri-stem'].value_counts()

/robots.txt                       305
/                                 284
/wp-login.php                      47
/admin/                            18
/ads.txt                           12
                                 ... 
/wp-admin/admin-ajax.php            1
/sitemap                            1
/wp-config.php.swp                  1
/contact.html/js/popper.min.js      1
/apple-touch-icon-152x152.png       1
Name: cs-uri-stem, Length: 182, dtype: int64

In [58]:
# quick counts of the 200/ok hits
df['ok'].describe()

count     2916
unique       2
top       True
freq      1628
Name: ok, dtype: object

In [ ]:
# install jupyter lab/dash prior to the next sections
# pipenv install jupyterlab==1.0 jupyterlab-dash==0.1.0a3
# jupyter labextension install jupyterlab-dash@0.1.0-alpha.3
# jupyter lab

In [11]:
# browse records as a dash table
import dash
import dash_table
import dash_html_components as html
import jupyterlab_dash

In [12]:
viewer = jupyterlab_dash.AppViewer()
app = dash.Dash(__name__)
app.layout = html.Div([
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True, "hideable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=True,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable="single",
        row_selectable="multi",
        row_deletable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 20,
        style_table={'overflowX': 'scroll',
                     'overflowY': 'scroll'},
        style_cell={
            'minWidth': '100px', 'maxWidth': '1080px',
            'whiteSpace': 'normal'
        },
        fixed_rows={ 'headers': True, 'data': 0 },
        style_data={
            'whiteSpace': 'normal',
            'height': 'auto'
        },        
        style_data_conditional=[
            {
                'if': {'row_index': 'odd'},
                'backgroundColor': 'rgb(248, 248, 248)'
            }
        ],
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold',
            'whiteSpace': 'normal',
            'height': 'auto',
            'overflow': 'hidden',
            'textOverflow': 'ellipsis'
            
        }        
    ),
    html.Div(id='datatable-interactivity-container')
])

viewer.show(app)

In [30]:
# get the URL for the dash table for use in another browser window if desired.
url=f'{viewer._jupyterlab_url}proxy/{viewer.port}'
print("A new window should open after running the previous cell, you can also view the table at:")
print(url)

A new window should open after running the previous cell, you can also view the table at:
http://localhost:8888/proxy/59358
